# Forecasting Bus Demand in Banana Republic Municipalities
### Prepared by: Said Bolluk
### E-Mail: said.bolluk@ozu.edu.tr

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit
%matplotlib inline

In [2]:
# Display model diagram
from sklearn import set_config
set_config(display='diagram')

# 1) Data Preprocessing

In [3]:
mydf = pd.read_csv("municipality_bus_utilization.csv")
mydf.head()

timestamp  municipality_id  usage  total_capacity
0  2017-06-04 07:59:42                9    454            1332
1  2017-06-04 07:59:42                8    556            2947
2  2017-06-04 07:59:42                4   1090            3893
3  2017-06-04 07:59:42                0    204            2813
4  2017-06-04 07:59:42                7    718            2019

Transform the "timestamp" column into a datatime object:

In [4]:
mydf["timestamp"] = pd.to_datetime(mydf["timestamp"])

In [5]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13070 entries, 0 to 13069
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestamp        13070 non-null  datetime64[ns]
 1   municipality_id  13070 non-null  int64         
 2   usage            13070 non-null  int64         
 3   total_capacity   13070 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 408.6 KB


Select and save the hour, day, and month info for tHe future calculations:

In [6]:
mydf["time_hour"] = mydf.timestamp.apply(lambda x: x.hour)
mydf["time_day"] = mydf.timestamp.apply(lambda x: x.day)
mydf["time_month"] = mydf.timestamp.apply(lambda x: x.month)

In [7]:
mydf.head()

timestamp  municipality_id  usage  total_capacity  time_hour  \
0 2017-06-04 07:59:42                9    454            1332          7   
1 2017-06-04 07:59:42                8    556            2947          7   
2 2017-06-04 07:59:42                4   1090            3893          7   
3 2017-06-04 07:59:42                0    204            2813          7   
4 2017-06-04 07:59:42                7    718            2019          7   

   time_day  time_month  
0         4           6  
1         4           6  
2         4           6  
3         4           6  
4         4           6

Dataset contains a single or couple of recordings for each hour. Therefore, I will take the mean of these recordings for each hour.

In [8]:
municipalities = list(range(0, 10))
months = [6, 7, 8]

empty_frame = pd.DataFrame()

for i in municipalities:
    
    data_muni = mydf[mydf["municipality_id"]==i]
    
    for j in months:
        
        data_month = data_muni[data_muni["time_month"]==j]
        
        day_list = np.sort(data_month["time_day"].value_counts().index.to_list())

        for k in day_list:
            
            data_day = data_month[data_month["time_day"]==k]
            
            # Group usage per hour by averaging out the recordings in the same hour
            aggregate = data_day.groupby(by="time_hour")["usage"].mean().round(2)
            
            # Define a new data frame to keep recordings per hour
            frame = {
                "Municipality": i,
                "Month": j,
                "Day": k,
                "Hour": aggregate.index, 
                "Average Usage": aggregate.values,
            }            
            reduced_data = pd.DataFrame(frame)
            empty_frame = empty_frame.append(reduced_data, ignore_index=True)

Create a copy of the empty_frame:

In [9]:
bus_demand = empty_frame.copy()

## Missing Data Imputation

Note that recordings are kept between the same dates in each municipality. Let us examine the missing days for each month:

In [10]:
months = [6, 7, 8]
total_day = 0

for i in months:
    
    day_list = np.sort(empty_frame[empty_frame["Month"]==i]["Day"].value_counts().index.to_list())
    total_day += len(day_list)
    
    print(f"Month-{i}: {day_list}")
    print(f"Total Day: {total_day}\n")

Month-6: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 22 23 24 25 26 27 28 29
 30]
Total Day: 25

Month-7: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
Total Day: 55

Month-8: [ 1  2  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Total Day: 72



There are two missing days (20, 21) for the Month 6, and two missing days (3, 4) for the Month 8. The Month 7 does not contain any missing days. Lut us create a row for with the average use per hour and complete the missing days of each month:

In [11]:
municipalities = list(range(0, 10))
missing_months = [6, 8]

missing_frame = pd.DataFrame()

for i in municipalities:
    
    data_muni = bus_demand[bus_demand["Municipality"]==i]
    
    for j in missing_months:
        
        if j==6:
            missing_days = [20, 21]
        
        elif j==8:
            missing_days = [3, 4]
        
        else:
            print("ERROR!")
            
        for k in missing_days:
            
            grouping = data_muni.groupby(by="Hour")["Average Usage"].mean()
            #print(i, j, k, grouping)
            frame = {
                "Municipality": i,
                "Month": j,
                "Day": k,
                "Hour": grouping.index, 
                "Average Usage": grouping.values,
            }            
            missing_data = pd.DataFrame(frame)
            missing_frame = missing_frame.append(missing_data, ignore_index=True)
            
final_df = pd.concat([bus_demand, missing_frame], axis=0, ignore_index=True)

In [12]:
print("Missing:", bus_demand.shape)
print("Completed:", final_df.shape)

Missing: (6740, 5)
Completed: (7140, 5)


Check if there are any missing days:

In [13]:
months = [6, 7, 8]

for i in months:
    
    day_list = np.sort(final_df[final_df["Month"]==i]["Day"].value_counts().index.to_list())
    
    print(f"Month-{i}: {day_list}")

Month-6: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30]
Month-7: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
Month-8: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Sort intances by Municipality, Month, Day, and Hour. Reset the index.

In [14]:
final_df = final_df.sort_values(by=["Municipality", "Month", "Day", "Hour"], ascending = [True, True, True, True])
final_df = final_df.reset_index()
final_df = final_df.drop(columns="index")

Combine the date features and present as: Month/Day/Hour

In [15]:
for i in range(final_df.shape[0]):
    
    m, d, h = final_df.loc[i, "Month"], final_df.loc[i, "Day"], final_df.loc[i, "Hour"]
    final_df.loc[i, "Date"] = f"{m}/{d}/{h}"

We will sort the final_df according to the month and day columns since the added rows disturbed the order of the original frame. We will then save the dataset to a new CSV file so we can use it quickly in the future:

In [16]:
final_df

Municipality  Month  Day  Hour  Average Usage     Date
0                0      6    4     7          204.0    6/4/7
1                0      6    4     8          289.5    6/4/8
2                0      6    4     9          457.0    6/4/9
3                0      6    4    10          558.5   6/4/10
4                0      6    4    11          605.5   6/4/11
...            ...    ...  ...   ...            ...      ...
7135             9      8   19    12          825.0  8/19/12
7136             9      8   19    13          918.5  8/19/13
7137             9      8   19    14          981.0  8/19/14
7138             9      8   19    15          965.0  8/19/15
7139             9      8   19    16          847.0  8/19/16

[7140 rows x 6 columns]

In [17]:
# Sort values
final_df = final_df.sort_values(by=["Municipality", "Month", "Day", "Hour"], ascending = [True, True, True, True])

# Re-order columns and round values
final_df = final_df[['Date', 'Month', 'Day', 'Hour', 'Municipality','Average Usage']].round(2)

# Save the dataset
final_df.to_csv("ready_data.csv", index=False)

In [18]:
final_df.sample(10)

Date  Month  Day  Hour  Municipality  Average Usage
1342  8/10/15      8   10    15             1         410.33
6927  7/27/13      7   27    13             9         922.50
5402   7/17/8      7   17     8             7         685.50
2699   8/3/12      8    3    12             3        1256.75
2661  7/29/12      7   29    12             3        1621.00
5122  6/17/10      6   17    10             7        1035.00
2696    8/3/9      8    3     9             3         899.01
1789  7/12/11      7   12    11             2         381.00
6362  8/13/10      8   13    10             8        1233.00
1539  6/15/15      6   15    15             2         501.00

Observe the average correlation between the target and features using the Pearson correlation method:

In [19]:
sum_array = np.zeros(3,)
for i in range(0, 10):
    
    correlation = final_df[final_df["Municipality"]==i].corr(method="pearson")["Average Usage"][0:3]
    sum_array = np.sum([sum_array, np.abs(correlation.values)], axis=0)
    
frame = {
    "Features": correlation.index.to_list(),
    "Average Correlation (absolute)": sum_array / 10,
}

corr_df = pd.DataFrame(frame)
corr_df

Features  Average Correlation (absolute)
0    Month                        0.129301
1      Day                        0.104097
2     Hour                        0.546100

# 2) Model Training

In [20]:
data = pd.read_csv("ready_data.csv")

In [21]:
data.head()

Date  Month  Day  Hour  Municipality  Average Usage
0   6/4/7      6    4     7             0          204.0
1   6/4/8      6    4     8             0          289.5
2   6/4/9      6    4     9             0          457.0
3  6/4/10      6    4    10             0          558.5
4  6/4/11      6    4    11             0          605.5

Display the columns of the dataset:

In [22]:
data.columns.to_list()

['Date', 'Month', 'Day', 'Hour', 'Municipality', 'Average Usage']

Define a dataset named bus_demand using a set of columns:

In [23]:
bus_demand = data[['Month', 'Day', 'Hour', 'Municipality', 'Average Usage']]
bus_demand.head()

Month  Day  Hour  Municipality  Average Usage
0      6    4     7             0          204.0
1      6    4     8             0          289.5
2      6    4     9             0          457.0
3      6    4    10             0          558.5
4      6    4    11             0          605.5

Try different regressors and gather the results under the report_df:

In [24]:
report_df = pd.DataFrame()

Import necessary libraries for machine learning models and regression evaluation metrics to perform time series regression over the bus_demand data:

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

## 2.1. Linear Regression

In [26]:
# Define a regression model
linear = LinearRegression()

In [27]:
r2_list = list()
mape_list = list()

for muni_id in range(0, 10):
    
    # Train and test split as last two weeks of each municipality are for testing purposes
    mapping_data = bus_demand[bus_demand["Municipality"]==muni_id].drop(columns=["Municipality"])
    test_mapping = (mapping_data["Month"]==8) & (mapping_data["Day"]>=4)
    
    training_set = mapping_data[~test_mapping]
    y_train = training_set["Average Usage"]
    X_train = training_set.drop(columns=["Average Usage"])    
    
    test_set = mapping_data[test_mapping]
    y_test = test_set["Average Usage"]
    X_test = test_set.drop(columns=["Average Usage"])
    
    # Train the model
    linear.fit(X_train, y_train)
    # Make predictions
    preds = linear.predict(X_test)
    
    r2 = r2_score(y_test, preds)
    r2_list.append(r2)
    mape = mean_absolute_percentage_error(y_test, preds)
    mape_list.append(mape)
    
# Calculate the mean score using metrics from each municipality
mean_r2 = np.mean(r2_list).round(4)
mean_mape = np.mean(mape_list).round(4)

lr_result = {
    "Model": "Linear Regression",
    "R2": mean_r2,
    "MAPE": mean_mape,
}

report_df = report_df.append(lr_result, ignore_index=True, sort=True)

## 2.2. Random Forest Regressor

This time we will perform a basic hyperparameter tuning for the forest model. We will analyze different tree depths and try to determine the optimal one that provides the least mean percentage error and the highest R2 Score.

In [28]:
depth_list = np.arange(1, 11)

overall_mape = list()
overall_r2 = list()

for depth in depth_list:
    
    # Define a regression model
    forest = RandomForestRegressor(
        max_depth=depth,
        random_state=42,
    )

    r2_list = list()
    mape_list = list()

    for muni_id in range(0, 10):

        # Train and test split as last two weeks of each municipality are for testing purposes
        mapping_data = bus_demand[bus_demand["Municipality"]==muni_id].drop(columns=["Municipality"])
        test_mapping = (mapping_data["Month"]==8) & (mapping_data["Day"]>=4)

        training_set = mapping_data[~test_mapping]
        y_train = training_set["Average Usage"]
        X_train = training_set.drop(columns=["Average Usage"])

        test_set = mapping_data[test_mapping]
        y_test = test_set["Average Usage"]
        X_test = test_set.drop(columns=["Average Usage"])

        # Train the model
        forest.fit(X_train, y_train)
        # Make predictions
        preds = forest.predict(X_test)

        r2 = r2_score(y_test, preds)
        r2_list.append(r2)
        mape = mean_absolute_percentage_error(y_test, preds)
        mape_list.append(mape)
    
    # Calculate the mean score using metrics from each municipality
    mean_r2 = np.mean(r2_list).round(4)
    mean_mape = np.mean(mape_list).round(4)
    
    # Save the average R2 Score and MAPE for a maximum tree depth:
    overall_r2.append(mean_r2)
    overall_mape.append(mean_mape)

Use a validation curve that helps us observe the trade-off between bias (MAPE) and variance (R2) and determine the optimal tree depth.

In [29]:
plt.figure(figsize=(8, 6))
plt.plot(depth_list, overall_r2, label="R2 Score")
plt.plot(depth_list, overall_mape, label="MAPE")
plt.xticks(depth_list)
plt.xlabel("Maximum tree depth")
plt.legend()
plt.show()

From the validation curve above, we can see that the optimal tree depth is **four (4)**. After this point, the result improvement slows down. We will now use this parameter value to train the Random Forest Regressor.

In [30]:
# Define a regression model
forest = RandomForestRegressor(
    max_depth=4,
    random_state=42,
)

In [31]:
r2_list = list()
mape_list = list()

for muni_id in range(0, 10):
    
    # Train and test split as last two weeks of each municipality are for testing purposes
    mapping_data = bus_demand[bus_demand["Municipality"]==muni_id].drop(columns=["Municipality"])
    test_mapping = (mapping_data["Month"]==8) & (mapping_data["Day"]>=4)
    
    training_set = mapping_data[~test_mapping]
    y_train = training_set["Average Usage"]
    X_train = training_set.drop(columns=["Average Usage"])
    
    test_set = mapping_data[test_mapping]
    y_test = test_set["Average Usage"]
    X_test = test_set.drop(columns=["Average Usage"])
    
    # Train the model
    forest.fit(X_train, y_train)
    # Make predictions
    preds = forest.predict(X_test)
    
    r2 = r2_score(y_test, preds)
    r2_list.append(r2)
    mape = mean_absolute_percentage_error(y_test, preds)
    mape_list.append(mape)
    
# Calculate the mean score using metrics from each municipality
mean_r2 = np.mean(r2_list).round(4)
mean_mape = np.mean(mape_list).round(4)

forest_result = {
    "Model": "Random Forest Regressor",
    "R2": mean_r2,
    "MAPE": mean_mape,
}

report_df = report_df.append(forest_result, ignore_index=True, sort=True)

## 2.3. Gradient Boosting Regressor

This time we will perform a relatively extensive hyperparameter tuning using more than one hyperparameter.

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
GradientBoostingRegressor().get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

Select a random municipality for simplicity:

In [34]:
random_muni = 5

Perform hyperparameter tuning and calculate the run time:

In [35]:
training_set = bus_demand[bus_demand["Municipality"]==random_muni].drop(columns=["Municipality"])
y_train = training_set["Average Usage"]
X_train = training_set.drop(columns=["Average Usage"])

# Initiate a regression model
gradient = GradientBoostingRegressor(random_state=42)

grid_params = {
    "loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "criterion": ["friedman_mse", "squared_error"],
    "learning_rate": np.arange(1, 110, 10)* 1e-3,
    "max_depth": np.arange(1, 11),    
}

grid_search = GridSearchCV(gradient, grid_params, verbose=1, cv=5, n_jobs=-1)

start = timeit.default_timer()

grid_search.fit(X_train, y_train)

stop = timeit.default_timer()
run_time = stop - start

Fitting 5 folds for each of 880 candidates, totalling 4400 fits


Display the run time for the grid search:

In [36]:
print(f"Run time (min): {(run_time/60):.2f}")

Run time (min): 15.56


According to the results of GridSearch, the best score and the optimal estimation parameters are:

In [37]:
print(f"Best Score: {grid_search.best_score_:.4f}")
print("Optimal parameter combination:\n", grid_search.best_estimator_)

Best Score: 0.5174
Optimal parameter combination:
 GradientBoostingRegressor(learning_rate=0.031, max_depth=2, random_state=42)


Let us now train a regression model using the optimal setting obtained from grid search:

In [38]:
# Define a regression model
gradient = GradientBoostingRegressor(
    criterion="squared_error",
    learning_rate=0.031,
    max_depth=2,
    random_state=42,
)

In [39]:
r2_list = list()
mape_list = list()

for muni_id in range(0, 10):
    
    # Train and test split as last two weeks of each municipality are for testing purposes
    mapping_data = bus_demand[bus_demand["Municipality"]==muni_id].drop(columns=["Municipality"])
    test_mapping = (mapping_data["Month"]==8) & (mapping_data["Day"]>=4)
    
    training_set = mapping_data[~test_mapping]
    y_train = training_set["Average Usage"]
    X_train = training_set.drop(columns=["Average Usage"])
    
    test_set = mapping_data[test_mapping]
    y_test = test_set["Average Usage"]
    X_test = test_set.drop(columns=["Average Usage"])
    
    # Train the model
    gradient.fit(X_train, y_train)
    # Make predictions
    preds = gradient.predict(X_test)
    
    r2 = r2_score(y_test, preds)
    r2_list.append(r2)
    mape = mean_absolute_percentage_error(y_test, preds)
    mape_list.append(mape)
    
# Calculate the mean score using metrics from each municipality
mean_r2 = np.mean(r2_list).round(4)
mean_mape = np.mean(mape_list).round(4)

gradient_result = {
    "Model": "Gradient Boosting Regressor",
    "R2": mean_r2,
    "MAPE": mean_mape,
}

report_df = report_df.append(gradient_result, ignore_index=True, sort=True)

# 3) Results and Discussion

Edit the order of columns and sort the the values:

In [40]:
report_df = report_df[["Model", "R2", "MAPE"]]
report_df = report_df.sort_values(by=["R2"], ascending=False)
report_df

Model      R2    MAPE
1      Random Forest Regressor  0.4356  0.1922
2  Gradient Boosting Regressor  0.4341  0.2047
0            Linear Regression  0.2864  0.2659

Save the results to a CSV file:

In [41]:
report_df.to_csv("regression_results.csv")

We can see that the **random forest regressor** outperformed the others in predicting the average bus demand of the given municipalities in terms of accuracy (relatively low error) and generalization power (relatively high R2). Let us now visualize the estimation process of this regressor by plotting the target and predicted values.

In [42]:
mapping_data = bus_demand[bus_demand["Municipality"]==5].reset_index()
date_list = list()

for i in range(mapping_data.shape[0]):
    
    m, d, h = mapping_data.loc[i, "Month"], mapping_data.loc[i, "Day"], mapping_data.loc[i, "Hour"]
    date_list.append(f"{m}/{d}/{h}")

In [43]:
# Define a regression model
final_model = RandomForestRegressor(
    max_depth=4,
    random_state=42,
)

# Select a random municipality
random_muni = 5
    
# Train and test split as last two weeks of each municipality are for testing purposes
mapping_data = bus_demand[bus_demand["Municipality"]==random_muni].drop(columns=["Municipality"]).reset_index()
date_list = list()

for i in range(mapping_data.shape[0]):
    
    m, d, h = mapping_data.loc[i, "Month"], mapping_data.loc[i, "Day"], mapping_data.loc[i, "Hour"]
    date_list.append(f"{m}/{d}/{h}")

# Apply mapping to split the data into training and test sets
test_mapping = (mapping_data["Month"]==8) & (mapping_data["Day"]>=4)

training_set = mapping_data[~test_mapping]
y_train = training_set["Average Usage"]
X_train = training_set.drop(columns=["Average Usage"])

test_set = mapping_data[test_mapping]
y_test = test_set["Average Usage"]
X_test = test_set.drop(columns=["Average Usage"])

# Train the model
final_model.fit(X_train, y_train)
# Make predictions
preds = final_model.predict(X_test)
# Evaluate the regression results
r2 = r2_score(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

# Visualize the estimation process
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(
    X_train.index, 
    y_train, 
    label="training",
    color="salmon",
    lw=2,
    linestyle="solid",
)
ax.plot(
    X_test.index, 
    y_test, 
    label="target_test",
    lw=2,
    color="black",
    alpha=0.3,
    linestyle='solid',
)
ax.plot(
    X_test.index, 
    final_model.predict(X_test), 
    label=f"predictions_test\nMAPE={mape:.4f}\nR2={r2:.4f}",
    color="violet",
    alpha=0.95,
    lw=2,
    linestyle='dashed',
)
ax.set_xlabel("Date (month/day/hour)")
ax.set_ylabel("Average Bus Demand (people)")
ax.set_xticklabels(date_list[::75], rotation="vertical", fontsize=12)
ax.legend()
plt.show()

C:\Users\User\anaconda3\envs\pytorch_p37\lib\site-packages\ipykernel_launcher.py:68: UserWarning: FixedFormatter should only be used together with FixedLocator


**Final Notes:** We see that the non-parametric forest regressor did not achieve great accuracy (36% error) nor properly represented the variance in the target (only %60 of the variance was represented). Even though the forest model performed better than the others thanks to its learning capacity over non-linear patterns of the features, the features in the dataset did not provide this model or the others with a great deal of information. Therefore, feature extraction is necessary to increase the accuracy and generalization power of the regressors trained over the bus demand dataset. Deriving or creating new features might help to understand the hidden non-linear relationship between the features and the target.

# END